In [11]:
import os
import json
import yt_dlp

class VideoDownloader:
    def __init__(self, base_dir="./downloads"):
        self.base_dir = base_dir
        self.setup_directories()

    def setup_directories(self):
        """기본 디렉토리 생성"""
        directories = {
            "youtube": ["audio", "subtitle", "video", "metadata"],
            "insta": ["video", "metadata"],
            "tiktok": ["video", "metadata"]
        }
        
        for platform, subdirs in directories.items():
            base_dir = os.path.join(self.base_dir, platform)
            for subdir in subdirs:
                dir_path = os.path.join(base_dir, subdir)
                os.makedirs(dir_path, exist_ok=True)
                setattr(self, f"{platform}_{subdir}_dir", dir_path)

    def fetch_metadata(self, url, platform):
        """메타데이터 추출 및 저장"""
        try:
            with yt_dlp.YoutubeDL({"quiet": True}) as ydl:
                info = ydl.extract_info(url, download=False)
                metadata_dir = getattr(self, f"{platform}_metadata_dir")
                metadata_file = os.path.join(metadata_dir, f"{info.get('id')}.json")
                
                with open(metadata_file, "w", encoding="utf-8") as f:
                    json.dump(info, f, indent=4, ensure_ascii=False)

                print(f"Metadata saved for URL: {url}")
                return info
        except Exception as e:
            print(f"Failed to fetch metadata for URL: {url}")
            print(f"Error: {e}")
            return None

    def download_content(self, url, platform, content_type):
        """콘텐츠 다운로드"""
        options = self.get_options(platform, content_type)
        try:
            self.fetch_metadata(url, platform)
            with yt_dlp.YoutubeDL(options) as ydl:
                ydl.download([url])
            print(f"{platform.capitalize()} {content_type} downloaded successfully for URL: {url}")
        except Exception as e:
            print(f"Failed to download {platform} {content_type} for URL: {url}")
            print(f"Error: {e}")

    def get_options(self, platform, content_type):
        """콘텐츠 유형에 따른 옵션 반환"""
        base_dir = getattr(self, f"{platform}_{content_type}_dir")
        base_options = {
            "outtmpl": os.path.join(base_dir, "%(title)s.%(ext)s"),
        }
        if content_type == "audio":
            base_options.update({
                "format": "bestaudio/best",
                "noplaylist": True,
                "postprocessors": [{"key": "FFmpegExtractAudio", "preferredcodec": "m4a"}],
            })
        elif content_type == "subtitle":
            base_options.update({
                "writesubtitles": True,
                "writeautomaticsub": True,
                "skip_download": True,
                "subtitlesformat": "vtt",
                "subtitleslangs": ["en", "ko", "auto"],
            })
        elif content_type in ["video"]:
            base_options.update({
                "format": "bestvideo+bestaudio/best",
                "merge_output_format": "mp4",
            })
        return base_options

In [12]:
downloader = VideoDownloader(base_dir='/home/jovyan/work/video-downloader/downloads')

# Youtube
url = "https://www.youtube.com/watch?v=Ox5e3ZXJtPM"
downloader.download_content(url, "youtube", "video") # 주어진 YouTube URL에서 비디오를 다운로드합니다.
downloader.download_content(url, "youtube", "audio") # 주어진 YouTube URL에서 오디오를 다운로드합니다.
downloader.download_content(url, "youtube", "subtitle") # 주어진 YouTube URL에서 자막을 다운로드합니다.

# Insta
url = "https://www.instagram.com/reel/DDgG9wiTWg7/"
downloader.download_content(url, "insta", "video") # 주어진 Instagram URL에서 비디오를 다운로드합니다.

# Tiktok
url = "https://www.tiktok.com/@hammerking75/video/7420376675633909010dd"
downloader.download_content(url, "tiktok", "video") # 주어진 TikTok URL에서 비디오를 다운로드합니다.

Metadata saved for URL: https://www.youtube.com/watch?v=Ox5e3ZXJtPM
[youtube] Extracting URL: https://www.youtube.com/watch?v=Ox5e3ZXJtPM
[youtube] Ox5e3ZXJtPM: Downloading webpage
[youtube] Ox5e3ZXJtPM: Downloading ios player API JSON
[youtube] Ox5e3ZXJtPM: Downloading mweb player API JSON
[youtube] Ox5e3ZXJtPM: Downloading m3u8 information
[info] Ox5e3ZXJtPM: Downloading 1 format(s): 401+251
[download] Destination: /home/jovyan/work/video-downloader/downloads/youtube/video/아이폰 AI 신기능 대폭 추가! iOS 18.2 새로운 변화점과 핵심 신기능 총정리.f401.mp4
[download] 100% of  542.12MiB in 00:00:42 at 12.86MiB/s     
[download] Destination: /home/jovyan/work/video-downloader/downloads/youtube/video/아이폰 AI 신기능 대폭 추가! iOS 18.2 새로운 변화점과 핵심 신기능 총정리.f251.webm
[download] 100% of   18.92MiB in 00:00:01 at 17.12MiB/s  
[Merger] Merging formats into "/home/jovyan/work/video-downloader/downloads/youtube/video/아이폰 AI 신기능 대폭 추가! iOS 18.2 새로운 변화점과 핵심 신기능 총정리.mp4"
Deleting original file /home/jovyan/work/video-downloader/downl